In [2]:
import random
import pandas as pd

random.seed(42)

# Function to generate a random cost based on the claim type and year
def generate_claim_cost(claim_type, year):
    if claim_type == 'Dentist':
        base_cost = 100
    elif claim_type == 'Psychiatrist':
        base_cost = 150
    elif claim_type == 'General Physician':
        base_cost = 80
    elif claim_type == 'Physiotherapy':
        base_cost = 120
    else:
        base_cost = 50

    # Adjust cost based on year
    if year == 2021:
        base_cost *= 1.2
    elif year == 2023:
        base_cost *= 1.5

    # Add some random variation
    cost = random.uniform(base_cost - 20, base_cost + 20)
    return round(cost, 2)

# Generating sample data
claim_types = ['Dentist', 'Psychiatrist', 'General Physician', 'Physiotherapy']
years = [2021, 2022, 2023]
people = ['John', 'Jane', 'Michael', 'Emily', 'William', 'Emma', 'Daniel', 'Olivia', 'Lucas', 'Ava']

data = []
for year in years:
    for person in people:
        num_claims = random.randint(1, 5)  # Random number of claims per person per year
        for _ in range(num_claims):
            claim_type = random.choice(claim_types)
            cost = generate_claim_cost(claim_type, year)
            date = pd.to_datetime(f"{random.randint(1, 12)}/{random.randint(1, 28)}/{year}", format='%m/%d/%Y')
            data.append([person, claim_type, date, year, cost])

# Create the DataFrame
columns = ['person', 'claim_type', 'date', 'year', 'amount']
claims_df = pd.DataFrame(data, columns=columns)
claims_df.sample(5)

,person,claim_type,date,year,amount
6,William,Psychiatrist,2021-06-04,2021,198.29
16,Olivia,Dentist,2021-03-18,2021,124.37
61,Emily,Physiotherapy,2023-05-07,2023,166.19
50,Ava,Psychiatrist,2022-11-14,2022,133.77
0,John,Dentist,2021-04-08,2021,129.66


## Sum

In [3]:
import locale
import numpy as np

locale.setlocale(locale.LC_MONETARY, 'en_US.UTF-8')
def fmt_currency(x):
    return 'N/A' if np.isnan(x) else locale.currency(x, grouping=True)

(
    claims_df.groupby('year')
    .agg({'amount': 'sum'})
    .assign(diff=lambda x: x['amount'].diff())
    .style.format(fmt_currency)
)

,amount,diff
year,,
2021,"$3,814.54",N/A
2022,"$2,890.29",-$924.25
2023,"$4,178.03","$1,287.74"


In [4]:
import ibis

ibis.options.interactive = True
claims = ibis.memtable(claims_df, name="claims")
claims

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓
┃ person  ┃ claim_type        ┃ date                ┃ year  ┃ amount  ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩
│ string  │ string            │ timestamp           │ int64 │ float64 │
├─────────┼───────────────────┼─────────────────────┼───────┼─────────┤
│ John    │ Dentist           │ 2021-04-08 00:00:00 │  2021 │  129.66 │
│ Jane    │ Dentist           │ 2021-09-03 00:00:00 │  2021 │  127.07 │
│ Jane    │ Physiotherapy     │ 2021-02-07 00:00:00 │  2021 │  125.27 │
│ Michael │ Dentist           │ 2021-12-21 00:00:00 │  2021 │  122.45 │
│ Michael │ Physiotherapy     │ 2021-10-09 00:00:00 │  2021 │  132.82 │
│ Emily   │ Psychiatrist      │ 2021-06-09 00:00:00 │  2021 │  187.93 │
│ William │ Psychiatrist      │ 2021-06-04 00:00:00 │  2021 │  198.29 │
│ William │ Dentist           │ 2021-06-28 00:00:00 │  2021 │  115.20 │
│ Emma    │ General Physician │ 2021-12-15 00:00:00 │  2021 │  108.29 │
│ Emma    │ Dentist           │ 2021-07-03 00:00:00 │  2021 │  138.92 │
│ …       │ …                 │ …                   │     … │       … │
└─────────┴───────────────────┴─────────────────────┴───────┴─────────┘

In [5]:
type(claims)

ibis.expr.types.relations.Table

In [17]:
import munpack

unpacker = munpack.SumUnpacker(
    fact='amount',
    period='year',
    dimensions=['claim_type']
)
unpack = unpacker(claims)
unpack.execute()

,claim_type,year,inner,mix
0,Dentist,2021,NaN,NaN
1,Dentist,2022,-170.700000,-311.240000
2,Dentist,2023,338.180000,480.330000
3,General Physician,2021,NaN,NaN
4,General Physician,2022,-95.053333,249.693333
5,General Physician,2023,313.151429,-236.051429
6,Psychiatrist,2021,NaN,NaN
7,Psychiatrist,2022,-282.030000,147.410000
8,Psychiatrist,2023,544.140000,-861.710000
9,Physiotherapy,2021,NaN,NaN


In [20]:
(
    unpack
    .execute()
    .groupby('year')
    .apply(lambda x: (x.inner + x.mix).sum())
    .to_frame('sum')
    .style.format(fmt_currency)
)

,sum
year,
2021,$0.00
2022,-$924.25
2023,"$1,287.74"


## Ratio

In [60]:
unpacker = munpack.MeanUnpacker(
    fact='amount',
    period='year',
    dimensions=['claim_type']
)
unpack = unpacker(claims)
ibis.to_sql(unpack)


```sql
WITH t0 AS (
  SELECT
    t7.claim_type AS claim_type,
    t7.year AS year,
    SUM(t7.amount) AS sum,
    COUNT(t7.amount) AS count
  FROM claims AS t7
  GROUP BY
    1,
    2
), t1 AS (
  SELECT
    t0.claim_type AS claim_type,
    t0.year AS year,
    t0.sum AS sum,
    t0.count AS count,
    t0.sum / t0.count AS ratio
  FROM t0
), t2 AS (
  SELECT
    t1.year AS year,
    SUM(t1.sum) AS sum_sum,
    SUM(t1.count) AS count_sum
  FROM t1
  GROUP BY
    1
), t3 AS (
  SELECT
    t1.claim_type AS claim_type,
    t1.year AS year,
    t1.sum AS sum,
    t1.count AS count,
    t1.ratio AS ratio,
    t2.year AS year_right,
    t2.sum_sum AS sum_sum,
    t2.count_sum AS count_sum
  FROM t1
  LEFT OUTER JOIN t2
    ON t1.year = t2.year
), t4 AS (
  SELECT
    t3.claim_type AS claim_type,
    t3.year AS year,
    t3.sum AS sum,
    t3.count AS count,
    t3.ratio AS ratio,
    t3.year_right AS year_right,
    t3.sum_sum AS sum_sum,
    t3.count_sum AS count_sum,
    t3.count / CAST(t3.count_sum AS DOUBLE) AS share,
    t3.sum_sum / t3.count_sum AS global_ratio
  FROM t3
), t5 AS (
  SELECT
    t4.claim_type AS claim_type,
    t4.year AS year,
    t4.sum AS sum,
    t4.count AS count,
    t4.ratio AS ratio,
    t4.year_right AS year_right,
    t4.sum_sum AS sum_sum,
    t4.count_sum AS count_sum,
    t4.share AS share,
    t4.global_ratio AS global_ratio,
    LAG(t4.ratio, CAST(1 AS TINYINT)) OVER (PARTITION BY t4.claim_type ORDER BY t4.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS ratio_lag,
    LAG(t4.share, CAST(1 AS TINYINT)) OVER (PARTITION BY t4.claim_type ORDER BY t4.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS share_lag,
    LAG(t4.global_ratio, CAST(1 AS TINYINT)) OVER (PARTITION BY t4.claim_type ORDER BY t4.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS global_ratio_lag
  FROM t4
)
SELECT
  t6.claim_type,
  t6.year,
  t6."inner",
  t6.mix
FROM (
  SELECT
    t5.claim_type AS claim_type,
    t5.year AS year,
    t5.sum AS sum,
    t5.count AS count,
    t5.ratio AS ratio,
    t5.year_right AS year_right,
    t5.sum_sum AS sum_sum,
    t5.count_sum AS count_sum,
    t5.share AS share,
    t5.global_ratio AS global_ratio,
    t5.ratio_lag AS ratio_lag,
    t5.share_lag AS share_lag,
    t5.global_ratio_lag AS global_ratio_lag,
    t5.share * (
      t5.ratio - t5.ratio_lag
    ) AS "inner",
    (
      t5.share - t5.share_lag
    ) * (
      t5.ratio_lag - t5.global_ratio_lag
    ) AS mix
  FROM t5
) AS t6
```

## Funnel

In [61]:
import pandas as pd

traffic = pd.DataFrame({
    'date': ['2018-01-01', '2018-01-01', '2018-01-01', '2019-01-01', '2019-01-01', '2019-01-01', '2018-02-01', '2018-02-01', '2018-02-01', '2019-02-01', '2019-02-01', '2019-02-01'],
    'group': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
    'impressions': [1000, 2000, 2500, 1000, 2150, 2000, 50, 2000, 2500, 2500, 2150, 2000],
    'clicks': [150, 150, 250, 120, 200, 400, 20, 300, 250, 1000, 323, 320],
    'conversions': [120, 150, 125, 100, 145, 166, 10, 150, 125, 500, 145, 166],
    'revenue': ['$8,600', '$9,400', '$10,750', '$9,055', '$8,739', '$10,147', '$500', '$11,400', '$8,750', '$50,000', '$10,739', '$12,147'],
})
traffic['date'] = pd.to_datetime(traffic['date'])
traffic['revenue'] = traffic['revenue'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)
traffic.style.format({'revenue': fmt_currency})

,date,group,impressions,clicks,conversions,revenue
0,2018-01-01 00:00:00,A,1000,150,120,"$8,600.00"
1,2018-01-01 00:00:00,B,2000,150,150,"$9,400.00"
2,2018-01-01 00:00:00,C,2500,250,125,"$10,750.00"
3,2019-01-01 00:00:00,A,1000,120,100,"$9,055.00"
4,2019-01-01 00:00:00,B,2150,200,145,"$8,739.00"
5,2019-01-01 00:00:00,C,2000,400,166,"$10,147.00"
6,2018-02-01 00:00:00,A,50,20,10,$500.00
7,2018-02-01 00:00:00,B,2000,300,150,"$11,400.00"
8,2018-02-01 00:00:00,C,2500,250,125,"$8,750.00"
9,2019-02-01 00:00:00,A,2500,1000,500,"$50,000.00"


In [62]:
class FunnelUnpacker:
    def __init__(self, funnel, period, dimensions):
        self.funnel = funnel
        self.period = period
        self.dimensions = dimensions

    def transform(self, facts):

        unpack = (
            facts.groupby([self.period, *self.dimensions])
            [self.funnel]
            .sum().sort_values(self.period)
        )

        ratios = {
            (f'{num}_by_{den}' if den else num): (num, den)
            for den, num in [(None, self.funnel[0]), *zip(self.funnel, self.funnel[1:])]
        }

        for ratio_name, (num, den) in ratios.items():
            if den:
                unpack[ratio_name] = unpack[num] / unpack[den]
            unpack[f'{ratio_name}_lag'] = unpack.groupby(self.dimensions)[ratio_name].shift(1)

        ratio_names = list(ratios)
        for i, _ in enumerate(ratio_names):
            before = ratio_names[:i]
            current = f'({ratio_names[i]} - {ratio_names[i]}_lag)'
            after = [f'{x}_lag' for x in ratio_names[i+1:]]
            formula = ' * '.join(filter(None, [*before, current, *after]))
            unpack[f'{ratio_names[i]}_contribution'] = unpack.eval(formula)

        return (
            unpack[[col for col in unpack.columns if '_contribution' in col]]
            .rename(columns=lambda x: x.replace('_contribution', ''))
            .dropna()
        )


unpacker = FunnelUnpacker(
    funnel=['impressions', 'clicks', 'conversions', 'revenue'],
    period='year',
    dimensions=['month', 'group']
)
traffic = traffic.assign(
    month=traffic.date.dt.month_name(),
    year=traffic.date.dt.year
)
unpack = unpacker.transform(traffic)
unpack.style.format(fmt_currency)

In [134]:
import functools
import operator

funnel = ['impressions', 'clicks', 'conversions', 'revenue']
period = 'year'
dimensions = ['month', 'group']

traffic_table = ibis.memtable(traffic, name="traffic")

# Sum events by period and dimensions
unpack = (
    traffic_table.group_by([period, *dimensions])
    .aggregate(**{step: traffic_table[step].sum() for step in funnel})
)
ratios = {
    (f'{num}_over_{den}' if den else num): (num, den)
    for den, num in [(None, funnel[0]), *zip(funnel, funnel[1:])]
}
ratio_names = list(ratios)

unpack = unpack.mutate(**{
    ratio_name: unpack[num] / unpack[den]
    for ratio_name, (num, den) in ratios.items()
    if den
})

unpack = unpack.group_by(dimensions).order_by(period).mutate(**{
    f'{ratio_name}_lag': unpack[ratio_name].lag(1)
    for ratio_name in ratios
})

unpack = unpack.mutate(**{
    f'{ratio_name}_contribution': functools.reduce(
        operator.mul,
        [
            *[unpack[ratio_name] for ratio_name in ratio_names[:i]],
            unpack[ratio_names[i]] - unpack[f"{ratio_names[i]}_lag"],
            *[unpack[f'{ratio_name}_lag'] for ratio_name in ratio_names[i+1:]]
        ]
    )
    for i, ratio_name in enumerate(ratio_names)
})

unpack = (
    unpack
    .order_by([period, *dimensions])
    .select([
        period,
        *dimensions,
        *[col for col in unpack.schema() if col.endswith('_contribution')]
    ])
    .dropna(how="any")
)

ibis.to_sql(unpack)
unpack.execute()

```sql
WITH t0 AS (
  SELECT
    t6.year AS year,
    t6.month AS month,
    t6."group" AS "group",
    SUM(t6.impressions) AS impressions,
    SUM(t6.clicks) AS clicks,
    SUM(t6.conversions) AS conversions,
    SUM(t6.revenue) AS revenue
  FROM traffic AS t6
  GROUP BY
    1,
    2,
    3
), t1 AS (
  SELECT
    t0.year AS year,
    t0.month AS month,
    t0."group" AS "group",
    t0.impressions AS impressions,
    t0.clicks AS clicks,
    t0.conversions AS conversions,
    t0.revenue AS revenue,
    t0.clicks / CAST(t0.impressions AS DOUBLE) AS clicks_over_impressions,
    t0.conversions / CAST(t0.clicks AS DOUBLE) AS conversions_over_clicks,
    t0.revenue / t0.conversions AS revenue_over_conversions
  FROM t0
), t2 AS (
  SELECT
    t1.year AS year,
    t1.month AS month,
    t1."group" AS "group",
    t1.impressions AS impressions,
    t1.clicks AS clicks,
    t1.conversions AS conversions,
    t1.revenue AS revenue,
    t1.clicks_over_impressions AS clicks_over_impressions,
    t1.conversions_over_clicks AS conversions_over_clicks,
    t1.revenue_over_conversions AS revenue_over_conversions,
    LAG(t1.impressions, CAST(1 AS TINYINT)) OVER (PARTITION BY t1.month, t1."group" ORDER BY t1.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS impressions_lag,
    LAG(t1.clicks_over_impressions, CAST(1 AS TINYINT)) OVER (PARTITION BY t1.month, t1."group" ORDER BY t1.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS clicks_over_impressions_lag,
    LAG(t1.conversions_over_clicks, CAST(1 AS TINYINT)) OVER (PARTITION BY t1.month, t1."group" ORDER BY t1.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS conversions_over_clicks_lag,
    LAG(t1.revenue_over_conversions, CAST(1 AS TINYINT)) OVER (PARTITION BY t1.month, t1."group" ORDER BY t1.year ASC ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS revenue_over_conversions_lag
  FROM t1
), t3 AS (
  SELECT
    t2.year AS year,
    t2.month AS month,
    t2."group" AS "group",
    t2.impressions AS impressions,
    t2.clicks AS clicks,
    t2.conversions AS conversions,
    t2.revenue AS revenue,
    t2.clicks_over_impressions AS clicks_over_impressions,
    t2.conversions_over_clicks AS conversions_over_clicks,
    t2.revenue_over_conversions AS revenue_over_conversions,
    t2.impressions_lag AS impressions_lag,
    t2.clicks_over_impressions_lag AS clicks_over_impressions_lag,
    t2.conversions_over_clicks_lag AS conversions_over_clicks_lag,
    t2.revenue_over_conversions_lag AS revenue_over_conversions_lag,
    (
      t2.impressions - t2.impressions_lag
    ) * t2.clicks_over_impressions_lag * t2.conversions_over_clicks_lag * t2.revenue_over_conversions_lag AS impressions_contribution,
    t2.impressions * (
      t2.clicks_over_impressions - t2.clicks_over_impressions_lag
    ) * t2.conversions_over_clicks_lag * t2.revenue_over_conversions_lag AS clicks_over_impressions_contribution,
    t2.impressions * t2.clicks_over_impressions * (
      t2.conversions_over_clicks - t2.conversions_over_clicks_lag
    ) * t2.revenue_over_conversions_lag AS conversions_over_clicks_contribution,
    t2.impressions * t2.clicks_over_impressions * t2.conversions_over_clicks * (
      t2.revenue_over_conversions - t2.revenue_over_conversions_lag
    ) AS revenue_over_conversions_contribution
  FROM t2
), t4 AS (
  SELECT
    t3.year AS year,
    t3.month AS month,
    t3."group" AS "group",
    t3.impressions AS impressions,
    t3.clicks AS clicks,
    t3.conversions AS conversions,
    t3.revenue AS revenue,
    t3.clicks_over_impressions AS clicks_over_impressions,
    t3.conversions_over_clicks AS conversions_over_clicks,
    t3.revenue_over_conversions AS revenue_over_conversions,
    t3.impressions_lag AS impressions_lag,
    t3.clicks_over_impressions_lag AS clicks_over_impressions_lag,
    t3.conversions_over_clicks_lag AS conversions_over_clicks_lag,
    t3.revenue_over_conversions_lag AS revenue_over_conversions_lag,
    t3.impressions_contribution AS impressions_contribution,
    t3.clicks_over_impressions_contribution AS clicks_over_impressions_contribution,
    t3.conversions_over_clicks_contribution AS conversions_over_clicks_contribution,
    t3.revenue_over_conversions_contribution AS revenue_over_conversions_contribution
  FROM t3
  ORDER BY
    t3.year ASC,
    t3.month ASC,
    t3."group" ASC
), t5 AS (
  SELECT
    t4.year AS year,
    t4.month AS month,
    t4."group" AS "group",
    t4.impressions_contribution AS impressions_contribution,
    t4.clicks_over_impressions_contribution AS clicks_over_impressions_contribution,
    t4.conversions_over_clicks_contribution AS conversions_over_clicks_contribution,
    t4.revenue_over_conversions_contribution AS revenue_over_conversions_contribution
  FROM t4
)
SELECT
  t5.year,
  t5.month,
  t5."group",
  t5.impressions_contribution,
  t5.clicks_over_impressions_contribution,
  t5.conversions_over_clicks_contribution,
  t5.revenue_over_conversions_contribution
FROM t5
WHERE
  NOT t5.year IS NULL
  AND NOT t5.month IS NULL
  AND NOT t5."group" IS NULL
  AND NOT t5.impressions_contribution IS NULL
  AND NOT t5.clicks_over_impressions_contribution IS NULL
  AND NOT t5.conversions_over_clicks_contribution IS NULL
  AND NOT t5.revenue_over_conversions_contribution IS NULL
```

In [118]:
ibis.selectors.endswith('_contribution')

Predicate(predicate=<function endswith.<locals>.<lambda> at 0x1571e4fe0>)

In [98]:
[*before, current, *after]

[┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
 ┃ Subtract(impressions, impressions_lag) ┃
 ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
 │ int64                                  │
 ├────────────────────────────────────────┤
 │                                   NULL │
 │                                      0 │
 │                                   NULL │
 │                                   2450 │
 │                                   NULL │
 │                                    150 │
 │                                   NULL │
 │                                   NULL │
 │                                    150 │
 │                                   NULL │
 │                                      … │
 └────────────────────────────────────────┘,
 ┏━━━━━━━━━━━━━━━━━━━━━━━━━━┓
 ┃ clicks / impressions_lag ┃
 ┡━━━━━━━━━━━━━━━━━━━━━━━━━━┩
 │ float64                  │
 ├──────────────────────────┤
 │                     NULL │
 │                     0.15 │
 │                     NULL │
 │         

In [96]:
unpack['(impressions - impressions_lag)']

IbisTypeError: Column '(impressions - impressions_lag)' is not found in table. Existing columns: 'year', 'month', 'group', 'impressions', 'clicks', 'conversions', 'revenue', 'clicks / impressions', 'conversions / clicks', 'revenue / conversions', 'impressions_lag', 'clicks / impressions_lag', 'conversions / clicks_lag', 'revenue / conversions_lag'.

In [75]:
{
    (f'{num}_over_{den}' if den else num): unpack[num] / unpack[den]
    for den, num in [(None, funnel[0]), *zip(funnel, funnel[1:])]
}

NotImplementedError: Selection rows or columns with NoneType objects is not supported